
* Работа для агенства "Digital Scouting"
* Данные по зарплатам игроков примерно 30 ведущих футбольных лиг.

# Содержание

1. [Подготовим данные и сделаем первые выводы](#view1)
2. [Зарплатный рейтинг по возврастам](#view5)
  - 2.1 [Итоговая таблица по возрастам](#age1)
3. [Зарплатный рейтинг по позициям](#view4)
  - 3.1 [Итоговая таблица по позициям](#poz1)
4. [Национальный рейтинг](#naz1)
  - 4.1 [Национальный рейтинг итоговая таблица](#nazf)
5. [Национальный рейтинг по зп](#naz2)
  - 5.1 [Национальный рейтинг по игрокам итоговая таблица](#nazf2)
6. [Рейтинг клубов по ЗП ведомости](#view2) 

 - 6.1 [Итог таблица перцентили](#final1)
 - 6.2 [Итог таблица по игрокам](#final2)
 
7. [Рейтинг лиг по зп ведомости](#view3)
  - 7.1 [Итоговая таблица по рейтингу лиг](#zp1)


8. [Вывод](#vivod1)

## 1.Подготовим данные и сделаем первые выводы<a name="view1"></a>

In [1]:
import pandas as pd
import json
from IPython.display import display
import sklearn
from sklearn.cluster import KMeans
import numpy as np
import math
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
data = pd.read_csv('data_capology.csv',sep = ",",low_memory=False)
data = data.replace("\'", '\"')

In [3]:
#увеличим вывод строк
pd.options.display.max_rows = 1000

In [4]:
display(data.head(12))
display(data.shape)

,player,net_pw,net_py,position,age,country,club,league
0,Kylian Mbappé,977432,50826480,F,23,France,PSG,Ligue 1
1,Lionel Messi,684203,35578536,F,34,Argentina,PSG,Ligue 1
2,Neymar,606008,31512418,F,29,Brazil,PSG,Ligue 1
3,Cristiano Ronaldo,579644,30141507,F,36,Portugal,Manchester United,Premier League
4,Gareth Bale,319034,16589763,F,32,Wales,Real Madrid,La Liga
5,Eden Hazard,293230,15247944,F,31,Belgium,Real Madrid,La Liga
6,Sergio Ramos,293230,15247944,D,35,Spain,PSG,Ligue 1
7,Frenkie de Jong,273681,14231414,M,24,Netherlands,Barcelona,La Liga
8,Kevin De Bruyne,265245,13792754,F,30,Belgium,Manchester City,Premier League
9,David de Gea,248783,12936735,K,31,Spain,Manchester United,Premier League


(14291, 8)

In [5]:
#проверим тип данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14291 entries, 0 to 14290
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   player    14291 non-null  object
 1   net_pw    14291 non-null  object
 2   net_py    14291 non-null  int64 
 3   position  14291 non-null  object
 4   age       14291 non-null  int64 
 5   country   14291 non-null  object
 6   club      14291 non-null  object
 7   league    14291 non-null  object
dtypes: int64(2), object(6)
memory usage: 893.3+ KB


In [6]:
# с помощью лямб функции поменяем тип данных с object  на  float для того что бы произвести расчеты
#data['net_py'] = data['net_py'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
#data['net_pw'] = data['net_pw'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')


## 2.Зарплатный рейтинг по возврастам <a name="view5"></a>
Возрастная категория - Максимальная зп - Перцентиль 90 -Перцентиль 80 -Перцентиль 60 -Перцентиль 40
- до 20					
21-23					
24-26					
27-29					
30-32					
от 33					
					

In [7]:
def age_group(age):
    if age <= 20:
        return('до 20')
    if 21 <= age <= 23:
        return('21-23')
    if 24 <= age <= 26:
        return('24-26')
    if 27 <= age <= 29:
        return('27-29')
    if 30 <= age <= 32:
        return('30-32')
    if age > 30:
        return('от 33')
    return 

In [8]:
data['age_group'] = data['age'].apply(age_group)
print(data['age_group'].value_counts())

21-23    3364
24-26    3205
27-29    2600
до 20    2123
30-32    1794
от 33    1205
Name: age_group, dtype: int64


In [9]:
age = data.groupby(['age_group']).agg({'net_py':'max'})
age.sort_values(by='net_py',ascending = False,inplace=True)
display(age.shape)

(6, 1)

In [10]:
perage_40 = data.groupby('age_group')['net_py'].quantile(0.4)
perage_60 = data.groupby('age_group')['net_py'].quantile(0.6)
perage_80 = data.groupby('age_group')['net_py'].quantile(0.8)
perage_90 = data.groupby('age_group')['net_py'].quantile(0.9)

data_perc_age = pd.DataFrame()
data_perc_age['percentil40'] = perage_40
data_perc_age['percentil60'] = perage_60
data_perc_age['percentil80'] = perage_80
data_perc_age['percentil90'] = perage_90
display(data_perc_age.shape)

(6, 4)

In [11]:
#соеденим таблицы 
rating_age = age.merge(data_perc_age ,left_on='age_group' ,right_on = 'age_group' )
display(rating_age.shape)

(6, 5)

## 2.1 Итоговая таблица по возрастам<a name="age1"></a>

In [12]:
#Рейтинг зп по возрастам
final_age = rating_age.rename({'net_py':'макс зп','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
final_age = final_age.sort_values(by=['макс зп'],ascending=False).reset_index()
final_age.groupby('age_group')
display(final_age.shape)
final_age

(6, 6)

,age_group,макс зп,перцентиль40,перцентиль60,перцентиль80,перцентиль90
0,21-23,50826480,48226.00,91488.00,213471.00,437108.00
1,от 33,35578536,182975.00,365951.00,753698.80,1502678.40
2,27-29,31512418,142314.00,294794.00,640414.00,1266414.90
3,30-32,16589763,172810.00,374751.00,823389.00,1493009.30
4,24-26,14231414,91488.00,200000.00,461520.80,1002298.00
5,до 20,6810748,24113.00,40661.00,91488.00,193141.00


In [13]:
data.sort_values(by='net_py',ascending = False)
data.head(541)

,player,net_pw,net_py,position,age,country,club,league,age_group
0,Kylian Mbappé,977432,50826480,F,23,France,PSG,Ligue 1,21-23
1,Lionel Messi,684203,35578536,F,34,Argentina,PSG,Ligue 1,от 33
2,Neymar,606008,31512418,F,29,Brazil,PSG,Ligue 1,27-29
3,Cristiano Ronaldo,579644,30141507,F,36,Portugal,Manchester United,Premier League,от 33
4,Gareth Bale,319034,16589763,F,32,Wales,Real Madrid,La Liga,30-32
5,Eden Hazard,293230,15247944,F,31,Belgium,Real Madrid,La Liga,30-32
6,Sergio Ramos,293230,15247944,D,35,Spain,PSG,Ligue 1,от 33
7,Frenkie de Jong,273681,14231414,M,24,Netherlands,Barcelona,La Liga,24-26
8,Kevin De Bruyne,265245,13792754,F,30,Belgium,Manchester City,Premier League,30-32
9,David de Gea,248783,12936735,K,31,Spain,Manchester United,Premier League,30-32


 ## Вывод:  
 * Игроки 21-23 лет имеют наивысшую зп в своих клубах с достаточно сильным отрывом. 
 * Наименее оплачеваемые игроки оказались в возрасте до 20 лет, что в целом логично.
 

## 3.Зарплатный рейтинг по позициям<a name="view4"></a>
Позиция - Максимальная зп - Перцентиль 90 -Перцентиль 80 -Перцентиль 60 -Перцентиль 40

In [14]:
pos = data.groupby(['position']).agg({'net_py':'max'})
pos.sort_values(by='net_py',ascending = False,inplace=True)
display(pos.shape)

(4, 1)

In [15]:
perpos_40 = data.groupby('position')['net_py'].quantile(0.4)
perpos_60 = data.groupby('position')['net_py'].quantile(0.6)
perpos_80 = data.groupby('position')['net_py'].quantile(0.8)
perpos_90 = data.groupby('position')['net_py'].quantile(0.9)

data_perc_pos = pd.DataFrame()
data_perc_pos['percentil40'] = perpos_40
data_perc_pos['percentil60'] = perpos_60
data_perc_pos['percentil80'] = perpos_80
data_perc_pos['percentil90'] = perpos_90
display(data_perc_pos.shape)

(4, 4)

In [16]:
#соеденим таблицы 
rating_pos = pos.merge(data_perc_pos ,left_on='position' ,right_on = 'position' )
display(rating_pos.shape)

(4, 5)

## 3.1 Итоговая таблица по позициям <a name="poz1"></a>

In [17]:
#Рейтинг зп по позициям
final_pos = rating_pos.rename({'position':'позиция','net_py':'макс зп','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
final_pos = final_pos.sort_values(by=['макс зп'],ascending=False).reset_index()
final_pos.groupby('position')
display(final_pos.shape)
final_pos

(4, 6)

,position,макс зп,перцентиль40,перцентиль60,перцентиль80,перцентиль90
0,F,50826480,80000.00,180849.00,490536.00,1016530.00
1,D,15247944,81322.00,172810.00,437108.00,813224.00
2,M,14231414,72340.00,180000.00,478215.20,914877.00
3,K,12936735,48226.00,101653.00,304959.00,609918.00


## 4. Национальный рейтинг <a name="naz1"></a>

* Национальность * Кол-во игроков в выборке * Зарабатывают больше 2 000 000 * Зарабатывают больше 1 000 000	Зарабатывают больше 600 000 *Зарабатывают больше 400 000 * Зарабатывают больше 200 000	* Зарабатывают больше 100 000

In [18]:
def player_group(net_py):
    if net_py >= 2000000:
         return('2000000')
    if net_py >= 1000000:
         return('1000000')
    if net_py >= 600000:
         return('600000')
    if net_py >= 400000:
         return('400000')
    if net_py >= 200000:
         return('200000')
    if net_py >= 100000:
         return('100000')
    return 

In [19]:
data['player_group'] = data['net_py'].apply(player_group)

In [20]:
dtt = data[['country','net_py','player_group']]
dtt.sort_values(by='net_py',ascending = False)
dtt.head(552)

,country,net_py,player_group
0,France,50826480,2000000
1,Argentina,35578536,2000000
2,Brazil,31512418,2000000
3,Portugal,30141507,2000000
4,Wales,16589763,2000000
5,Belgium,15247944,2000000
6,Spain,15247944,2000000
7,Netherlands,14231414,2000000
8,Belgium,13792754,2000000
9,Spain,12936735,2000000


In [21]:
zar = data.groupby(['country']).agg({'player':'count'})
zar.sort_values(by='player',ascending = False,inplace=True)
display(zar.shape)

(153, 1)

In [22]:
zar_test = data.groupby(['country']).agg({'player':'count','player_group':'count'})
zar_test.sort_values(by='player',ascending = False,inplace=True)
display(zar_test.shape)

(153, 2)

In [23]:
#zar_test_2 = data.groupby(['country','player_group','net_py']).agg({'player':'count'})
#zar_test_2.sort_values(by='player',ascending = False,inplace=True)
#display(zar_test_2.shape)
#zar_test_2

zar_test_2 = data.groupby(['country','player_group']).agg({'net_py':'count'})
#zar_test_2.sort_values(by='player',ascending = False,inplace=True)
display(zar_test_2.shape)

(543, 1)

In [24]:
gpoup_pivot = pd.pivot_table(zar_test_2,
               index="country",
               values="net_py",
               columns="player_group"              
               ).reset_index()

#right_pivot.sort_values(by='player_group',ascending = False)

gpoup_pivot=gpoup_pivot.fillna(0)
display(gpoup_pivot.shape)

(140, 7)

In [25]:
play = gpoup_pivot.merge(zar ,left_on='country' ,right_on = 'country')
display(play.shape)
play = play.sort_values(by='player',ascending = False)

(140, 8)

## 4.1. Национальный рейтинг итоговая табица<a name="nazf"></a>

In [26]:
play = play.rename({'country':'Cтрана','player':'Кол-во игроков в выборке'}, axis=1)
display(play.shape)
play.reset_index()
play.head()

(140, 8)

,Cтрана,100000,1000000,200000,2000000,400000,600000,Кол-во игроков в выборке
43,England,206.00,84.00,159.00,56.00,57.00,57.00,1532
15,Brazil,153.00,71.00,171.00,50.00,87.00,63.00,1078
47,France,164.00,41.00,102.00,55.00,47.00,45.00,926
117,Spain,172.00,65.00,171.00,65.00,61.00,52.00,883
4,Argentina,204.00,35.00,171.00,22.00,93.00,57.00,860


In [27]:
play['more100_000'] = play[['100000','1000000','200000','2000000','400000','600000']].sum( axis=1)
play['more200_000'] = play[['1000000','200000','2000000','400000','600000']].sum( axis=1)
play['more400_000'] = play[['1000000','2000000','400000','600000']].sum( axis=1)
play['more600_000'] = play[['1000000','2000000','600000']].sum( axis=1)
play['more1_000_000'] = play[['1000000','2000000']].sum( axis=1)
play['more2_000_000'] = play[['2000000']].sum( axis=1)
play

,Cтрана,100000,1000000,200000,2000000,400000,600000,Кол-во игроков в выборке,more100_000,more200_000,more400_000,more600_000,more1_000_000,more2_000_000
43,England,206.00,84.00,159.00,56.00,57.00,57.00,1532,619.00,413.00,254.00,197.00,140.00,56.00
15,Brazil,153.00,71.00,171.00,50.00,87.00,63.00,1078,595.00,442.00,271.00,184.00,121.00,50.00
47,France,164.00,41.00,102.00,55.00,47.00,45.00,926,454.00,290.00,188.00,141.00,96.00,55.00
117,Spain,172.00,65.00,171.00,65.00,61.00,52.00,883,586.00,414.00,243.00,182.00,130.00,65.00
4,Argentina,204.00,35.00,171.00,22.00,93.00,57.00,860,582.00,378.00,207.00,114.00,57.00,22.00
26,Colombia,129.00,11.00,93.00,3.00,30.00,14.00,838,280.00,151.00,58.00,28.00,14.00,3.00
51,Germany,155.00,43.00,127.00,34.00,59.00,52.00,754,470.00,315.00,188.00,129.00,77.00,34.00
67,Italy,115.00,56.00,130.00,36.00,58.00,56.00,714,451.00,336.00,206.00,148.00,92.00,36.00
93,Netherlands,58.00,25.00,46.00,16.00,25.00,21.00,413,191.00,133.00,87.00,62.00,41.00,16.00
133,United States,62.00,6.00,52.00,5.00,24.00,13.00,403,162.00,100.00,48.00,24.00,11.00,5.00


In [28]:
play_drop=play.drop(['100000','1000000','200000','2000000','400000','600000'], axis=1)
play_drop

,Cтрана,Кол-во игроков в выборке,more100_000,more200_000,more400_000,more600_000,more1_000_000,more2_000_000
43,England,1532,619.00,413.00,254.00,197.00,140.00,56.00
15,Brazil,1078,595.00,442.00,271.00,184.00,121.00,50.00
47,France,926,454.00,290.00,188.00,141.00,96.00,55.00
117,Spain,883,586.00,414.00,243.00,182.00,130.00,65.00
4,Argentina,860,582.00,378.00,207.00,114.00,57.00,22.00
26,Colombia,838,280.00,151.00,58.00,28.00,14.00,3.00
51,Germany,754,470.00,315.00,188.00,129.00,77.00,34.00
67,Italy,714,451.00,336.00,206.00,148.00,92.00,36.00
93,Netherlands,413,191.00,133.00,87.00,62.00,41.00,16.00
133,United States,403,162.00,100.00,48.00,24.00,11.00,5.00


In [29]:
teto = data.query('country == "England"')
teto =teto.sort_values(by='net_py',ascending = False)
teto.head(57).reset_index()

,index,player,net_pw,net_py,position,age,country,club,league,age_group,player_group
0,11,Jadon Sancho,232090,12068660,F,21,England,Manchester United,Premier League,21-23,2000000
1,25,Raheem Sterling,198934,10344565,F,27,England,Manchester City,Premier League,27-29,2000000
2,24,Jack Grealish,198934,10344565,F,26,England,Manchester City,Premier League,24-26,2000000
3,39,John Stones,165778,8620471,D,27,England,Manchester City,Premier League,27-29,2000000
4,65,Marcus Rashford,132623,6896377,F,24,England,Manchester United,Premier League,24-26,2000000
5,64,Harry Kane,132623,6896377,F,28,England,Tottenham,Premier League,27-29,2000000
6,77,Ben Chilwell,125899,6546735,D,25,England,Chelsea,Premier League,24-26,2000000
7,78,Harry Maguire,125899,6546735,D,28,England,Manchester United,Premier League,27-29,2000000
8,83,Trent Alexander-Arnold,119407,6209151,D,23,England,Liverpool,Premier League,21-23,2000000
9,103,Luke Shaw,99467,5172283,D,26,England,Manchester United,Premier League,24-26,2000000


## 5. Национальный рейтинг по зп <a name="naz2"></a>

Национальность	Наивысшая зарплата	Зарплата 10 игрока	Зарплата 30 игрока	Зарплата 50 игрока	Зарплата 100 игрока	Зарплата 200 игрока	Зарплата 300 игрока

In [30]:
zp = data.groupby(['country']).agg({'net_py':'max'})
zp.sort_values(by='net_py',ascending = False,inplace=True)
display(zp.shape)
zp.head(1)

(153, 1)

,net_py
country,
France,50826480


In [31]:
filter_list=[10,30,50,100,200,300]

In [32]:
#data_coutry = data.copy()
#data_coutry['rank'] = data_coutry.groupby('country')['net_py'].rank(method='dense',ascending=False)
#data_coutry_filtered = data_coutry.query("rank == @filter_list")
#data_coutry_filtered.sort_values(by = 'rank', ascending = False)
#display(data_coutry_filtered.shape)
#data_coutry_filtered.head(305)


data['rank'] = data.groupby('country')['net_py'].rank(method='dense',ascending=False)
data_coutry_filtered = data.query("rank == @filter_list")
data_coutry_filtered.sort_values(by = 'rank', ascending = False)
display(data_coutry_filtered.shape)
data_coutry_filtered.head(305)

(305, 11)

,player,net_pw,net_py,position,age,country,club,league,age_group,player_group,rank
40,Anthony Martial,165778,8620471,F,25,France,Manchester United,Premier League,24-26,2000000,10.00
81,Cesc Fàbregas,122375,6363475,M,34,Spain,Monaco,Ligue 1,от 33,2000000,10.00
109,Kai Havertz,99467,5172283,F,22,Germany,Chelsea,Premier League,21-23,2000000,10.00
132,Casemiro,92465,4808185,M,29,Brazil,Real Madrid,La Liga,27-29,2000000,10.00
138,Tammy Abraham,87969,4574383,F,24,England,Roma,Serie A,24-26,2000000,10.00
246,Vitolo,63338,3293556,F,32,Spain,Getafe,La Liga,30-32,2000000,30.00
252,Rodrigo de Paul,62556,3252895,M,27,Argentina,Atletico Madrid,La Liga,27-29,2000000,10.00
259,Gelson Martins,61187,3181738,F,26,Portugal,Monaco,Ligue 1,24-26,2000000,10.00
261,Thomas Lemar,60992,3171572,F,26,France,Atletico Madrid,La Liga,24-26,2000000,30.00
271,Leander Dendoncker,59587,3098547,M,26,Belgium,Wolverhampton,Premier League,24-26,2000000,10.00


In [33]:
data_c = data.query('country == "France"')
data_c

,player,net_pw,net_py,position,age,country,club,league,age_group,player_group,rank
0,Kylian Mbappé,977432,50826480,F,23,France,PSG,Ligue 1,21-23,2000000,1.00
12,Raphaël Varane,225366,11719018,D,28,France,Manchester United,Premier League,27-29,2000000,2.00
13,Karim Benzema,225200,11710421,F,34,France,Real Madrid,La Liga,от 33,2000000,3.00
26,Antoine Griezmann,195486,10165296,F,30,France,Atletico Madrid,La Liga,30-32,2000000,4.00
27,Presnel Kimpembe,195486,10165296,D,26,France,PSG,Ligue 1,24-26,2000000,4.00
31,Paul Pogba,192210,9994924,M,28,France,Manchester United,Premier League,27-29,2000000,5.00
32,N'Golo Kanté,191978,9982867,M,30,France,Chelsea,Premier League,30-32,2000000,6.00
33,Ousmane Dembélé,187667,9758684,F,24,France,Barcelona,La Liga,24-26,2000000,7.00
34,Lucas Hernández,186494,9697692,D,25,France,Bayern Munich,1.Bundesliga,24-26,2000000,8.00
38,Kingsley Coman,176133,9158932,F,25,France,Bayern Munich,1.Bundesliga,24-26,2000000,9.00


In [34]:
data_pivot = pd.pivot_table(data_coutry_filtered,
               index='country',
               values="net_py",
               columns="rank"
               )

display(data_pivot.shape)

(84, 5)

In [35]:
zar = data.groupby(['country']).agg({'player':'count'})
zar.sort_values(by='player',ascending = False,inplace=True)
display(zar.shape)

(153, 1)

## 5.1. Национальный рейтинг по игрокам итоговая табица<a name="nazf2"></a>

In [36]:
rating_naz = data_pivot.merge(zp ,left_on='country' ,right_on = 'country')
display(rating_naz.shape)
rating_naz = rating_naz.sort_values(by='net_py',ascending = False)
rating_naz = rating_naz.fillna(0)
rating_naz 

(84, 6)

,10.0,30.0,50.0,100.0,200.0,net_py
country,,,,,,
France,8620471.00,3171572.00,1844660.00,660744.00,0.00,50826480
Argentina,3252895.00,1423141.00,1016530.00,540000.00,0.00,35578536
Brazil,4808185.00,2390000.00,1670000.00,792893.00,203306.00,31512418
Portugal,3181738.00,1301158.00,542547.00,38128.00,0.00,30141507
Wales,904245.00,192906.00,0.00,0.00,0.00,16589763
Spain,6363475.00,3293556.00,2155043.00,914877.00,50826.00,15247944
Belgium,3098547.00,864050.00,355785.00,0.00,0.00,15247944
Netherlands,2592170.00,1321488.00,681075.00,132623.00,0.00,14231414
Poland,1118183.00,349641.00,60283.00,0.00,0.00,12391496


## Вывод:
* По максимально зарплате в клубе лидеры Франция, Аргентина,Бразилия и Португалия, думаю можно без проблем называть этих игроков. Франция сильно в отрыве по игрокам из всех корзин. 

* Только у Бразили и Испании оказались 200е и 100е игроки у которых больши разброс по зп.

* Нули представлены здесь как замена пропущенным значениям, не все игроки  дотягивают до отметки 100е и 200е и ранк кидает их в другую корзину. Для того что бы проверить это оставил вычесления выше. 

* Получается, что несмотря на количество игроков зп могут быть сильно в разных диапозонах.

##  Найдем суммы зарплат 
(платежные ведомости) всех команд. 
* найти где граница между топ-клубами и просто сильными командами.

In [37]:
data_new = pd.read_csv('Capology_vedomosti.csv',sep = ",",low_memory=False)
data_new = data_new.replace("\'", '\"')

In [38]:
# с помощью лямб функции поменяем тип данных с object  на  float для того что бы произвести расчеты
#data_new['net_py'] = data_new['net_py'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
#data_new['net_pw'] = data_new['net_pw'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')



In [39]:
# в расчет берем зп за год
total_salary = data_new.groupby(['club']).agg({'net_py':'sum'})
total_salary.sort_values(by='net_py',ascending = False,inplace=True)

In [40]:
total_salary.reset_index()

,club,net_py
0,PSG,240333009
1,Manchester United,163547819
2,Real Madrid,157389280
3,Bayern Munich,128184381
4,Chelsea,115478143
5,Barcelona,114644205
6,Juventus,102262875
7,Manchester City,99485060
8,Liverpool,93089032
9,Inter Milan,85693444


## Вывод:
По таблице можно сказать что суммы от 48569786 млн евро зарплатной ведомости - это топ клубы, после этой отметки категория сильные клубы. ПСЖ вне конкуренции - сильный отрыв по зп. Остальные топ 4 имеют примерно одинаковую зп ведомость, между остальными топ клубами разница существенная.

Наш топ 5 довольно очевидный:
* PSG	240439745
* Manchester United	163547819
* Real Madrid	157389280
* Bayern Munich	128184381
* Chelsea 116225653

## Посмотрим на таблицу более подробно
Нужно найти:

* где будут российские клубы, Аякс, Спортинг, Зальцбург, Шахтер 

* Где будет первый клуб МЛС, первый мексиканский и т.д.

In [41]:
#найдем сумму платежной ведомости клуба
total_sum_test = data_new.groupby(['club','league']).agg({'net_py':'sum'})
total_sum_test.sort_values(by='net_py',ascending = False,inplace=True)
total_sum_test.head(3)

,,net_py
club,league,
PSG,Ligue 1,240333009
Manchester United,Premier League,163547819
Real Madrid,La Liga,157389280


In [42]:
#найдем максимальную зп
total_sum_2 = data_new.groupby(['club','league']).agg({'net_py':'max'})
total_sum_2.sort_values(by='net_py',ascending = False,inplace=True)

In [43]:
test_two = total_sum_test.merge(total_sum_2,left_on=['club','league'],right_on=['club','league'],how='right')
display(test_two.shape)

(488, 2)

In [44]:
total = test_two.rename({'net_py_x':'net_py','net_py_y':'net_pw'}, axis=1)

In [45]:
display(total.shape)
display(total.head(4))



(488, 2)

,,net_py,net_pw
club,league,,
PSG,Ligue 1,240333009,50826480
Manchester United,Premier League,163547819,30141507
Real Madrid,La Liga,157389280,16589763
Barcelona,La Liga,114644205,14231414


In [46]:
#сохраняем таблицу с чтобы отправить себе и потом загрузим снова с индексами
total.to_csv(r'total_ver2.csv')

In [47]:
#проставим индексы места в рейтинге
dt = pd.read_csv('total_ver2.csv',sep = ",",index_col = 0,low_memory=False)
dt = dt.replace("\'", '\"')

In [48]:
dt.groupby('club')
dt = dt.sort_values(by='net_py',ascending = False)
dt.reset_index()

,club,league,net_py,net_pw
0,PSG,Ligue 1,240333009,50826480
1,Manchester United,Premier League,163547819,30141507
2,Real Madrid,La Liga,157389280,16589763
3,Bayern Munich,1.Bundesliga,128184381,12391496
4,Chelsea,Premier League,115478143,11212641
5,Barcelona,La Liga,114644205,14231414
6,Juventus,Serie A,102262875,8132237
7,Manchester City,Premier League,99485060,13792754
8,Liverpool,Premier League,93089032,7583603
9,Inter Milan,Serie A,85693444,7115707


* первый клуб не из топ 5 это Бешикташ 26 место, следом Зенит 33 место.
* Фламенго 42 место * Спартак  - 48 место * Краснодар - 68 место * Аякс - 63 место  * Локомотив - 66 место * ЦСКА- 72 место * Динамо - 74 место * Рейнджерс - 77 место * Мексиканский UANL - 80 место * Орландо - 91 место *  Арсенал Тула - 114 место * Спортинг - 117 * Ростов - 122 * Сочи - 123 * Рубин - 135 * Ахмат - 143 * * Базель - 160 * Уфа 182 * Нижний - 206 * Урал - 217  * Крылья - 275 

## 6.Рейтинг клубов по ЗП ведомости<a name="view2"></a>

Задача: Собрать таблицу с колонками

- место в рейтинге - клуб - место в лиге по зп - размер ведомости net - максимальная зп - Перцентиль 90	- Перцентиль 80 - Перцентиль 60	- Перцентиль 40

Шаги:
1.Собрать зп ведомость по лигам и распределить клубы 
2.Собрать размезмер зп ведомости (сделано)
3.Найти максимальную зп
4.Найти перцентили и добавить в таблицу

In [49]:
percentil_40 = data_new.groupby('club')['net_py'].quantile(0.4)
percentil_60 = data_new.groupby('club')['net_py'].quantile(0.6)
percentil_80 = data_new.groupby('club')['net_py'].quantile(0.8)
percentil_90 = data_new.groupby('club')['net_py'].quantile(0.9)

In [50]:
#соберем перцентили
data_perc = pd.DataFrame()
data_perc['percentil40'] = percentil_40
data_perc['percentil60'] = percentil_60
data_perc['percentil80'] = percentil_80 
data_perc['percentil90'] = percentil_90
display(data_perc.shape)

(488, 4)

In [51]:
#максимальная зарпатная ведомость по команде
maxzp = data_new.groupby(['club','league']).agg({'net_pw':'max'})
maxzp.sort_values(by='net_pw',ascending = False,inplace=True)
display(maxzp.shape)
maxzp 

(488, 1)

,,net_pw
club,league,
Cardiff,Championship,9970
Karagumrukspor,Super Lig,9970
Bournemouth,Championship,9970
Granada,La Liga,9970
Celtic,Premiership,9970
Hoffenheim,1.Bundesliga,9970
Newcastle,Premier League,9970
Nottingham Forest,Championship,9970
Huddersfield,Championship,9970


In [52]:
#соеденим таблицы
test_2 = maxzp.merge(dt,left_on='club',right_on='club',how='right')
display(test_2.shape)
test_2 

(488, 4)

,net_pw_x,league,net_py,net_pw_y
club,,,,
PSG,977432,Ligue 1,240333009,50826480
Manchester United,99467,Premier League,163547819,30141507
Real Madrid,92465,La Liga,157389280,16589763
Bayern Munich,977,1.Bundesliga,128184381,12391496
Chelsea,99467,Premier League,115478143,11212641
Barcelona,97743,La Liga,114644205,14231414
Juventus,97743,Serie A,102262875,8132237
Manchester City,99467,Premier League,99485060,13792754
Liverpool,9970,Premier League,93089032,7583603


In [53]:
#соеденим таблицы
df_merge = test_2.merge(data_perc,left_on='club' ,right_on = 'club' )
#df_merge.sort_values(by=['net_py'],ascending=False).reset_index()
df_merge.groupby('club')
display(df_merge.shape)
df_merge 

(488, 8)

,net_pw_x,league,net_py,net_pw_y,percentil40,percentil60,percentil80,percentil90
club,,,,,,,,
PSG,977432,Ligue 1,240333009,50826480,527578.60,4159639.00,8050914.00,10673560.80
Manchester United,99467,Premier League,163547819,30141507,2963513.00,4205343.00,8340758.20,11753982.20
Real Madrid,92465,La Liga,157389280,16589763,4116945.00,4808185.00,10704057.00,11344470.50
Bayern Munich,977,1.Bundesliga,128184381,12391496,1813488.80,4525590.20,9697692.00,10857552.90
Chelsea,99467,Premier League,115478143,11212641,3421663.60,4135415.00,5172283.00,7599276.90
Barcelona,97743,La Liga,114644205,14231414,1500397.60,2976398.60,5133474.40,8579509.60
Juventus,97743,Serie A,102262875,8132237,3049589.00,4066118.00,5895872.00,6810748.00
Manchester City,99467,Premier League,99485060,13792754,3098547.00,4135415.00,5312139.40,9827336.80
Liverpool,9970,Premier League,93089032,7583603,2141253.00,3448188.00,5587030.20,6346596.20


In [54]:
df_merge = df_merge.drop(['net_pw_x'],axis=1)

In [55]:
final = df_merge
#.rename({'club':'клуб','net_pw_y':'макс зп','ligue':'лига','net_py':'размер ведомости','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
display(final.shape)
final.head(2)

(488, 7)

,league,net_py,net_pw_y,percentil40,percentil60,percentil80,percentil90
club,,,,,,,
PSG,Ligue 1,240333009,50826480,527578.60,4159639.00,8050914.00,10673560.80
Manchester United,Premier League,163547819,30141507,2963513.00,4205343.00,8340758.20,11753982.20


In [56]:
#создадим фильтр лист для ранк
filter_list_rank=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]

In [57]:
dt_ranked = final.copy()
dt_ranked['rank'] = final.groupby('league')['net_py'].rank(method='dense',ascending=False)
dt_rank_filtered = dt_ranked.query("rank == @filter_list_rank")
#dt_rank_filtered.sort_values(by = ['net_py'], ascending = False)
dt_rank_filtered.head(1)

,league,net_py,net_pw_y,percentil40,percentil60,percentil80,percentil90,rank
club,,,,,,,,
PSG,Ligue 1,240333009,50826480,527578.60,4159639.00,8050914.00,10673560.80,1.00


### 6.1 Итог таблица перцентили<a name="final1"></a>

In [58]:
dt_rank_filtered = dt_rank_filtered.rename({'club':'клуб','ligue':'лига','net_pw_y':'макс зп','net_py':'размер ведомости','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
dt_rank_filtered = dt_rank_filtered.sort_values(by = 'размер ведомости', ascending = False)
dt_rank_filtered

,league,размер ведомости,макс зп,перцентиль40,перцентиль60,перцентиль80,перцентиль90,rank
club,,,,,,,,
PSG,Ligue 1,240333009,50826480,527578.60,4159639.00,8050914.00,10673560.80,1.00
Manchester United,Premier League,163547819,30141507,2963513.00,4205343.00,8340758.20,11753982.20,1.00
Real Madrid,La Liga,157389280,16589763,4116945.00,4808185.00,10704057.00,11344470.50,1.00
Bayern Munich,1.Bundesliga,128184381,12391496,1813488.80,4525590.20,9697692.00,10857552.90,1.00
Chelsea,Premier League,115478143,11212641,3421663.60,4135415.00,5172283.00,7599276.90,2.00
Barcelona,La Liga,114644205,14231414,1500397.60,2976398.60,5133474.40,8579509.60,2.00
Juventus,Serie A,102262875,8132237,3049589.00,4066118.00,5895872.00,6810748.00,1.00
Manchester City,Premier League,99485060,13792754,3098547.00,4135415.00,5312139.40,9827336.80,3.00
Liverpool,Premier League,93089032,7583603,2141253.00,3448188.00,5587030.20,6346596.20,4.00


### Теперь соберем таблицу по игрокам

In [59]:
# отсортируем лиги по зп
total_ligue = dt.groupby(['league']).agg({'net_py':'sum'})
total_ligue.sort_values(by='net_py',ascending = False,inplace=True)
total_ligue.head(2)

,net_py
league,
Premier League,1076895758
La Liga,670741783


In [60]:
filter_list=[1,3,7,12,18]

#df_new.query("rank == @filter_list")

In [61]:
data_ranked = data_new.copy()
data_ranked['rank'] = data_new.groupby('club')['net_py'].rank(method='dense',ascending=False)
data_rank_filtered = data_ranked.query("rank == @filter_list")
display(data_rank_filtered.shape)
data_rank_filtered.sort_values(by = ['club','net_py'], ascending = False)
data_rank_filtered.head(1)

(3268, 9)

,player,net_pw,net_py,posotion,age,country,club,league,rank
0,Kylian Mbappé,977432,50826480,F,23,France,PSG,Ligue 1,1.00


In [62]:
statistic_pivot = pd.pivot_table(data_rank_filtered  ,
               index=['league','club'],
               values="net_py",
               columns="rank"
               )

display(statistic_pivot.shape)
statistic_pivot.head(2)

(488, 5)

rank                                1.00      3.00      7.00      12.00  \
league       club                                                         
1.Bundesliga Arminia Bielefeld  650579.00 487934.00 203306.00 111818.00   
             Augsburg          1128348.00 813224.00 487934.00 315124.00   

rank                               18.00  
league       club                         
1.Bundesliga Arminia Bielefeld       NaN  
             Augsburg          223637.00

In [63]:
final_player = statistic_pivot.merge(dt_rank_filtered,left_on=['league','club'],right_on=['league','club'],how='left').reset_index()
final_player = final_player[['league','club', 'rank','размер ведомости' ,1.0, 3.0, 7.0, 12.0, 18.0]]
#final_player = final_player.sort_values(by = ['net_py'], ascending = False)
final_player = final_player.fillna(0)
final_player 

,league,club,rank,размер ведомости,1.0,3.0,7.0,12.0,18.0
0,1.Bundesliga,Arminia Bielefeld,18.00,4930166,650579.00,487934.00,203306.00,111818.00,0.00
1,1.Bundesliga,Augsburg,11.00,12721868,1128348.00,813224.00,487934.00,315124.00,223637.00
2,1.Bundesliga,Bayer Leverkusen,5.00,26516173,1728100.00,1606117.00,1240166.00,874215.00,599752.00
3,1.Bundesliga,Bayern Munich,1.00,128184381,12391496.00,11049677.00,9158932.00,3771325.00,1026695.00
4,1.Bundesliga,Bochum,15.00,6937815,701405.00,498100.00,396447.00,233802.00,101653.00
5,1.Bundesliga,Borussia Dortmund,2.00,48610448,5926368.00,4737028.00,2155043.00,1402811.00,569257.00
6,1.Bundesliga,Eintracht Frankfurt,8.00,19141255,1504464.00,1412976.00,1016530.00,752232.00,386281.00
7,1.Bundesliga,Freiburg,16.00,6236411,508265.00,437108.00,304959.00,203306.00,50826.00
8,1.Bundesliga,Furth,17.00,6150005,569257.00,437108.00,325289.00,162645.00,101653.00
9,1.Bundesliga,Hertha Berlin,9.00,17962076,1687439.00,1402811.00,813224.00,528595.00,289711.00


## 6.2.Итог таблица по игрокам<a name="final2"></a>

In [64]:
final_player_2 = final_player.rename({'league':'лига','club':'клуб','rank':'ранг','net_py':'размер ведомости',1.0:'зп 1 игрока',3.0:'зп 3 игрока',7.0:'зп 7 игрока',12.0:'зп 12 игрока',18.0:'зп 18 игрока'}, axis=1)
final_player_2 = final_player.sort_values(by = 'размер ведомости', ascending = False)
final_player_2.reset_index()

,index,league,club,rank,размер ведомости,1.0,3.0,7.0,12.0,18.0
0,299,Ligue 1,PSG,1.00,240333009,50826480.00,31512418.00,8050914.00,5031822.00,2754795.00
1,364,Premier League,Manchester United,1.00,163547819,30141507.00,12068660.00,8270830.00,4310235.00,2592170.00
2,192,La Liga,Real Madrid,1.00,157389280,16589763.00,11710421.00,6404136.00,4574383.00,3252895.00
3,3,1.Bundesliga,Bayern Munich,1.00,128184381,12391496.00,11049677.00,9158932.00,3771325.00,1026695.00
4,357,Premier League,Chelsea,2.00,115478143,11212641.00,9380037.00,4135415.00,2760962.00,747509.00
5,180,La Liga,Barcelona,2.00,114644205,14231414.00,10165296.00,5336780.00,3049589.00,1463803.00
6,427,Serie A,Juventus,1.00,102262875,8132237.00,7115707.00,4066118.00,1524794.00,0.00
7,363,Premier League,Manchester City,3.00,99485060,13792754.00,8620471.00,4183641.00,2760962.00,470208.00
8,362,Premier League,Liverpool,4.00,93089032,7583603.00,6209151.00,3448188.00,2001396.00,434037.00
9,426,Serie A,Inter Milan,2.00,85693444,7115707.00,6099178.00,3557854.00,2033059.00,1016529.00


In [83]:
qq = data_new.query('club =="PSG"')
qq.sort_values(by = 'rank',ascending = False)

#а data_new?

KeyError: 'rank'

In [65]:
club = data_new.groupby(['club','league','player']).agg({'net_py':'sum'})
club.sort_values(by='net_py',ascending = False,inplace=True)
club.head(160)

net_py
club              league         player                             
PSG               Ligue 1        Kylian Mbappé              50826480
                                 Lionel Messi               35578536
                                 Neymar                     31512418
Manchester United Premier League Cristiano Ronaldo          30141507
Real Madrid       La Liga        Gareth Bale                16589763
PSG               Ligue 1        Sergio Ramos               15247944
Real Madrid       La Liga        Eden Hazard                15247944
Barcelona         La Liga        Frenkie de Jong            14231414
Manchester City   Premier League Kevin De Bruyne            13792754
Manchester United Premier League David de Gea               12936735
Bayern Munich     1.Bundesliga   Robert Lewandowski         12391496
Manchester United Premier League Jadon Sancho               12068660
                                 Raphaël Varane             11719018
Real Madrid       La Liga        Karim Benzema              11710421
Bayern Munich     1.Bundesliga   Manuel Neuer               11313974
Chelsea           Premier League Romelu Lukaku              11212641
Bayern Munich     1.Bundesliga   Thomas Müller              11049677
Real Madrid       La Liga        David Alaba                10978520
Bayern Munich     1.Bundesliga   Leroy Sané                 10775214
Barcelona         La Liga        Sergio Busquets            10734553
Real Madrid       La Liga        Marcelo                    10704057
                                 Toni Kroos                 10704057
Bayern Munich     1.Bundesliga   Joshua Kimmich             10510916
Manchester City   Premier League Jack Grealish              10344565
                                 Raheem Sterling            10344565
PSG               Ligue 1        Presnel Kimpembe           10165296
Atletico Madrid   La Liga        Antoine Griezmann          10165296
PSG               Ligue 1        Gianluigi Donnarumma       10165296
Barcelona         La Liga        Jordi Alba                 10165296
Atletico Madrid   La Liga        Jan Oblak                  10144965
Manchester United Premier League Paul Pogba                  9994924
Chelsea           Premier League N'Golo Kanté                9982867
Barcelona         La Liga        Ousmane Dembélé             9758684
Bayern Munich     1.Bundesliga   Leon Goretzka               9697692
                                 Lucas Hernández             9697692
Real Madrid       La Liga        Luka Modric                 9606205
Chelsea           Premier League Timo Werner                 9380037
Bayern Munich     1.Bundesliga   Kingsley Coman              9158932
Manchester United Premier League Edinson Cavani              8620471
Manchester City   Premier League John Stones                 8620471
Manchester United Premier League Bruno Fernandes             8270830
Juventus          Serie A        Matthijs de Ligt            8132237
PSG               Ligue 1        Achraf Hakimi               8132237
                                 Marco Verratti              8050914
                                 Marquinhos                  8050914
Atletico Madrid   La Liga        Koke                        7806947
                                 Luis Suárez                 7623972
Liverpool         Premier League Virgil van Dijk             7583603
Juventus          Serie A        Paulo Dybala                7420666
PSG               Ligue 1        Ángel Di María              7380005
Juventus          Serie A        Adrien Rabiot               7115707
Inter Milan       Serie A        Alexis Sánchez              7115707
AC Milan          Serie A        Zlatan Ibrahimovic          7115707
PSG               Ligue 1        Georginio Wijnaldum         7115707
Atletico Madrid   La Liga        João Félix                  7115707
Zenit             Premier Liga   Malcom                      7075046
Monaco            Ligue 1        Wissam Ben Yedder          

## 7.Рейтинг лиг по зп ведомости<a name="view3"></a>
Место в рейтинге - Лига	- Максимальная зп -  Перцентиль 90 -Перцентиль 80 -Перцентиль 60 -Перцентиль 40

In [66]:
liga = data_new.groupby(['league']).agg({'net_py':'max'})
liga.sort_values(by='net_py',ascending = False,inplace=True)
display(liga.shape)

(25, 1)

In [67]:
percent_40 = data_new.groupby('league')['net_py'].quantile(0.4)
percent_60 = data_new.groupby('league')['net_py'].quantile(0.6)
percent_80 = data_new.groupby('league')['net_py'].quantile(0.8)
percent_90 = data_new.groupby('league')['net_py'].quantile(0.9)

data_perc_ligue = pd.DataFrame()
data_perc_ligue['percentil40'] = percent_40
data_perc_ligue['percentil60'] = percent_60
data_perc_ligue['percentil80'] = percent_80 
data_perc_ligue['percentil90'] = percent_90
display(data_perc_ligue.shape)

(25, 4)

In [68]:
#соеденим таблицы 
rating_ligue = liga.merge(data_perc_ligue ,left_on='league' ,right_on = 'league' )
display(rating_ligue.shape)

(25, 5)

## 7.1 Итоговая таблица по рейтингу лиг<a name="zp1"></a>

In [69]:
#Рейтинг лиг по зп ведомостям
final_ligue = rating_ligue.rename({'league':'лига','net_py':'макс зп','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
final_ligue = final_ligue.sort_values(by=['макс зп'],ascending=False).reset_index()
final_ligue.groupby('league')
display(final_ligue.shape)
final_ligue



(25, 6)

,league,макс зп,перцентиль40,перцентиль60,перцентиль80,перцентиль90
0,Ligue 1,50826480,121984.00,274463.00,599752.00,1209670.00
1,Premier League,30141507,882543.00,1724094.00,2760962.00,4135415.00
2,La Liga,16589763,315124.00,630248.00,1463803.00,3067886.00
3,1.Bundesliga,12391496,223637.00,453372.00,914877.00,1597984.20
4,Serie A,8132237,406612.00,813224.00,1524794.00,2541324.00
5,Premier Liga,7075046,304959.00,548926.00,853885.00,1242199.00
6,Super Lig,5082648,172810.00,345620.00,677009.00,1075488.00
7,Brasileiro,3900000,50000.00,150000.00,330000.00,582000.00
8,Liga MX,3760000,100000.00,210000.00,362000.00,500000.00
9,MLS,3600000,79200.00,165216.00,330000.00,506398.80


In [70]:
mls = data_new.query('league == "MLS"')

In [71]:
mls

,player,net_pw,net_py,posotion,age,country,club,league
201,Chicharito,69231,3600000,F,33,Mexico,LA Galaxy,MLS
266,Gonzalo Higuaín,"58846,2",3060000,F,33,Argentina,Inter Miami,MLS
320,Carlos Vela,"51922,8",2700000,F,32,Mexico,LA FC,MLS
405,Alejandro Pozuelo,"43846,2",2280000,F,30,Spain,Toronto FC,MLS
438,Luiz Araújo,"41538,6",2160000,F,25,Brazil,Atlanta United,MLS
457,Josef Martínez,"40384,8",2100000,F,28,Venezuela,Atlanta United,MLS
698,Sebastián Driussi,29670,1542852,F,25,Argentina,Austin,MLS
756,Maxi Moralez,"28846,2",1500000,F,34,Argentina,NYC FC,MLS
781,Victor Wanyama,"27692,4",1440000,M,30,Kenya,Montreal Impact,MLS
803,Adrien Hunou,"27115,2",1410000,F,27,France,Minnesota United,MLS
